# AI agent with tools and Chatgpt

In [2]:
from langchain_core.tools import StructuredTool
from pydantic import BaseModel


def write_report(filename, html):
    with open(filename, 'w') as f:
        f.write(html)


class WriteReportArgsSchema(BaseModel):
    filename: str
    html: str


write_report_tool = StructuredTool.from_function(
    name="write_report",
    description="Write an HTML file to disk. Use this tool whenever someone asks for a report.",
    func=write_report,
    args_schema=WriteReportArgsSchema
)


In [6]:
import sqlite3
from pydantic import BaseModel
from typing import List
from langchain_core.tools import Tool

conn = sqlite3.connect("db.sqlite", check_same_thread=False)


def list_tables():
    c = conn.cursor()
    c.execute("SELECT name FROM sqlite_master WHERE type='table';")
    rows = c.fetchall()
    return "\n".join(row[0] for row in rows if row[0] is not None)


def run_sqlite_query(query):
    c = conn.cursor()
    try:
        c.execute(query)
        return c.fetchall()
    except sqlite3.OperationalError as err:
        return f"The following error occured: {str(err)}"


class RunQueryArgsSchema(BaseModel):
    query: str


run_query_tool = Tool.from_function(
    name="run_sqlite_query",
    description="Run a sqlite query.",
    func=run_sqlite_query,
    args_schema=RunQueryArgsSchema
)


def describe_tables(table_names):
    c = conn.cursor()
    placeholders = ', '.join(['?' for _ in table_names])
    query = f"SELECT sql FROM sqlite_master WHERE type='table' and name IN ({placeholders})"
    rows = c.execute(query, table_names)
    return '\n'.join(row[0] for row in rows if row[0] is not None)


class DescribeTablesArgsSchema(BaseModel):
    tables_names: List[str]


describe_tables_tool = Tool.from_function(
    name="describe_tables",
    description="Given a list of table names, returns the schema of those tables",
    func=describe_tables,
    args_schema=DescribeTablesArgsSchema
)


In [9]:
from langchain_core.callbacks.base import BaseCallbackHandler
from langchain_core.messages import (
    SystemMessage,
    HumanMessage,
    AIMessage,
    AIMessageChunk,
    ToolMessage,
)
from pyboxen import boxen


def boxen_print(*args, **kwargs):
    # simple wrapper so you can pass the same args as before
    print(boxen(*args, **kwargs))


class ChatModelStartHandler(BaseCallbackHandler):
    def _get_text(self, message):
        """
        Return a printable text for a message's content.
        Some providers may set content to a string or a structured object.
        We try to stringify safely.
        """
        content = getattr(message, "content", "")
        try:
            # if content is a dict/other, stringify nicely
            if isinstance(content, (str, bytes)):
                return content
            return str(content)
        except Exception:
            return "<unprintable content>"

    def _get_tool_calls(self, message):
        """
        Return a list of tool call dict-like objects in a uniform shape.
        Prefer message.tool_calls (new API), fall back to additional_kwargs.
        """
        # new standardized attribute
        tool_calls = getattr(message, "tool_calls", None)
        if tool_calls:
            return tool_calls

        # backwards-compat: some providers put tool calls in additional_kwargs
        additional = getattr(message, "additional_kwargs", None) or {}
        return additional.get("tool_calls") or additional.get("function_call") or []

    def on_chat_model_start(self, serialized, messages, **kwargs):
        # messages is usually a list of lists: [[SystemMessage, HumanMessage, ...], ...]
        print("\n\n\n\n========= Sending Messages =========\n\n")
        # iterate through the first batch (like your original code did)
        if not messages:
            return

        for message in messages[0]:
            # System messages
            if isinstance(message, SystemMessage):
                boxen_print(self._get_text(message), title="system", color="yellow")

            # Human / user messages
            elif isinstance(message, HumanMessage):
                boxen_print(self._get_text(message), title="human", color="green")

            # AI message (including chunks)
            elif isinstance(message, (AIMessage, AIMessageChunk)):
                # If the model produced tool calls, use the standardized attribute
                tool_calls = self._get_tool_calls(message)
                if tool_calls:
                    for tc in tool_calls:
                        # tc may be a dict-like or object depending on provider; handle both
                        try:
                            # dict-like
                            name = tc.get("name") or tc.get("function_name") or getattr(tc, "name", None)
                            args = tc.get("args") or tc.get("arguments") or getattr(tc, "args", None)
                        except Exception:
                            name = getattr(tc, "name", "<unknown>")
                            args = getattr(tc, "args", "<unknown>")

                        boxen_print(
                            f"Running tool {name} with args {args}",
                            title="ai (tool_call)",
                            color="cyan",
                        )
                else:
                    # plain AI content
                    boxen_print(self._get_text(message), title="ai", color="blue")

            # Tool messages (results of tool execution)
            elif isinstance(message, ToolMessage):
                boxen_print(self._get_text(message), title="tool", color="purple")

            # Fallback: try to print content if present
            else:
                text = self._get_text(message)
                if text:
                    boxen_print(text, title=message.__class__.__name__)


In [10]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.agents.middleware import before_model
from langgraph.checkpoint.memory import InMemorySaver
from dotenv import load_dotenv

load_dotenv()

handler = ChatModelStartHandler()
chat = ChatOpenAI(callbacks=[handler])

tables = list_tables()

# Create agent with new API
agent = create_agent(
    model=chat,
    tools=[run_query_tool, describe_tables_tool, write_report_tool],
    system_prompt=(
        "You are an AI that has access to a SQLite database.\n"
        f"The database has tables of: {tables}\n"
        "Do not make any assumptions about what tables exist "
        "or what columns exist. Instead, use the 'describe_tables' function"
    ),
    checkpointer=InMemorySaver(),
)

# Use the agent with thread_id for conversation history
config = {"configurable": {"thread_id": "default"}}

print("Initial query:")
initial_result = agent.invoke(
    {"messages": [{"role": "user", "content": "How many orders are there? Write the result to an html report."}]},
    config=config
)
print(f"Response: {initial_result['messages'][-1].content}")

print("\nFollowup query:")
followup_result = agent.invoke(
    {"messages": [{"role": "user", "content": "Repeat the exact same process for users."}]},
    config=config
)
print(f"Response: {followup_result['messages'][-1].content}")

Initial query:




========= Sending Messages =========




╭─ system ────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ You are an AI that has access to a SQLite database.                                                             │
│ The database has tables of: users                                                                               │
│ addresses                                                                                                       │
│ products                                                                                                        │
│ carts                                                                                                           │
│ orders                                                                                                          │
│ order_products                                                                                                  │
│ Do not make any assumptions about what tables exist or what columns ex

╭─ human ──────────────────────────────────────────────────────╮                                                   
│How many orders are there? Write the result to an html report.│                                                   
╰──────────────────────────────────────────────────────────────╯                                                   





========= Sending Messages =========




╭─ system ────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ You are an AI that has access to a SQLite database.                                                             │
│ The database has tables of: users                                                                               │
│ addresses                                                                                                       │
│ products                                                                                                        │
│ carts                                                                                                           │
│ orders                                                                                                          │
│ order_products                                                                                                  │
│ Do not make any assumptions about what tables exist or what columns ex

╭─ human ──────────────────────────────────────────────────────╮                                                   
│How many orders are there? Write the result to an html report.│                                                   
╰──────────────────────────────────────────────────────────────╯                                                   



...

╭─ human ────────────────────────────────╮                                                                         
│Repeat the exact same process for users.│                                                                         
╰────────────────────────────────────────╯                                                                         



╭─ ai (tool_call) ──────────────────────────────────────────────────────────────╮                                  
│Running tool run_sqlite_query with args {'query': 'SELECT COUNT(*) FROM users'}│                                  
╰───────────────────────────────────────────────────────────────────────────────╯                                  



╭─ tool ─╮                                                                                                         
│[[2000]]│                                                                                                         
╰────────╯                                                                                                         



╭─ ai (tool_call) ────────────────────────────────────────────────────────────────────────────────────────────────╮
│Running tool write_report with args {'filename': 'users_report.html', 'html': '<h1>Number of Users</h1><p>There  │
│are 2000 users in the database.</p>'}                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯



╭─ too─╮                                                                                                           
│ null │                                                                                                           
╰──────╯                                                                                                           

Response: I have generated a report showing that there are 2000 users in the database. You can download the report from the following link: [users_report.html](users_report.html)
